In [1]:
# IMPORTANT : si l'execution déclenche une erreur et demande nltk.download, 
# faire executer ces 2 lignes : 
# nltk.download('punkt')
# nltk.download("stopwords")

In [1]:
import requests
import nltk
import re
import numpy as np

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from string import punctuation
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest
import nltk

#---------------------------------------------
# FONCTION
# input : director name (case insensitive)
# output : cleaned article retrieved
#---------------------------------------------
def GetWikipediaSummary(director):
    # Récupération de la page exacte wikipedia (après redirection) à partir du nom de l'auteur non normalisé
    director.replace(" ","+")
    mysearchURL = "https://en.wikipedia.org/w/index.php?search="+director
    mysummaryURL = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&redirects=true&exintro&explaintext&titles="
    http = requests.get(mysearchURL)
    #On vire les 30 premiers caractères pour trouver le nom normalisé de l'auteur
    exactURL = mysummaryURL + http.url[30:]
    #Récupération du résumé
    fullresults= requests.get(exactURL).text
    summaryindex = fullresults.find("extract")
    summary =fullresults[summaryindex+10:-5]

    #CLEANUP
    summary = summary.lower()
    
    #On vire les requetes qui n'ont rien donné sur wikipedia
    if summary.find("may refer")!=-1:
        return ""
    if summary.find("normalized")!=-1:
        return ""
    
    #on vire les caractères spéciaux & les chiffres & les slashs
    summary.encode('ascii', 'replace')
    summary=re.sub("[0-9\\\\]+","",summary)

    
    #On vire le nom/prénom du directeur
    for director_part in director.lower().split(" "):
        summary = summary.replace(director_part,"")
    
    return summary

In [2]:
# Le plan
# 1. Récupérer pour les N réalisateurs les mieux notés en moyenne
# 2. Appliquer une pondération TF-IDF (Term Frequency / Inverse Document Frequency)
# 3. Lancer un algo de clustering

#Step 1
df = pd.read_csv("500 favorite directors.csv", sep=";", low_memory=False, names = ["director_name"])
df.set_index("director_name",inplace=True)

In [3]:
#on vire les index dupliqués éventuellement
df = df[~df.index.duplicated(keep='first')]

In [4]:
#Step 2
for x in range(len(df)):
    if type(df.iloc[x].name)!=str:
        df.loc[df.iloc[x].name,"summary"]=""
    else:
        summary = GetWikipediaSummary(df.iloc[x].name)
        print("Retrieving ("+str(x)+ "/" + str(len(df)) +") summary for : "+df.iloc[x].name)
        df.loc[df.iloc[x].name,"summary"]=summary
#On vire les entrées sans résumé
df_backup = df.copy()

Retrieving (0/590) summary for : 
Yoon-ki Lee
Retrieving (1/590) summary for : Abdellatif Kechiche
Retrieving (2/590) summary for : Abel Gance
Retrieving (3/590) summary for : Agnès Varda
Retrieving (4/590) summary for : Aki Kaurismäki
Retrieving (5/590) summary for : Akira Kurosawa
Retrieving (6/590) summary for : Alain Cavalier
Retrieving (7/590) summary for : Alain Resnais
Retrieving (8/590) summary for : Alain Robbe-Grillet
Retrieving (9/590) summary for : Alan Crosland
Retrieving (10/590) summary for : Albert Lamorisse
Retrieving (11/590) summary for : Albert Lewin
Retrieving (12/590) summary for : Alejandro Amenábar
Retrieving (13/590) summary for : Alejandro González Iñárritu
Retrieving (14/590) summary for : Alejandro Jodorowsky
Retrieving (15/590) summary for : Aleksa Gajic, Nebojsa Andric, Stevan Djordjevic
Retrieving (16/590) summary for : Aleksandr Dovzhenko
Retrieving (17/590) summary for : Aleksandr Sokurov
Retrieving (18/590) summary for : Alex Garland
Retrieving (19/590

Retrieving (165/590) summary for : Fred M. Wilcox
Retrieving (166/590) summary for : Fred Newmayer
Retrieving (167/590) summary for : Fred Niblo
Retrieving (168/590) summary for : Fred Zinnemann
Retrieving (169/590) summary for : Fritz Lang
Retrieving (170/590) summary for : Gabriel Axel
Retrieving (171/590) summary for : Gabriela Cowperthwaite
Retrieving (172/590) summary for : Garry Marshall
Retrieving (173/590) summary for : Georg Wilhelm Pabst 
Retrieving (174/590) summary for : George A. Romero
Retrieving (175/590) summary for : George Cukor
Retrieving (176/590) summary for : George Marshall
Retrieving (177/590) summary for : George Miller
Retrieving (178/590) summary for : George Roy Hill
Retrieving (179/590) summary for : Georges Cukor
Retrieving (180/590) summary for : Georges Franju
Retrieving (181/590) summary for : Georges Lautner
Retrieving (182/590) summary for : Georges Lucas
Retrieving (183/590) summary for : Georges Meliès
Retrieving (184/590) summary for : Georges Pere

Retrieving (331/590) summary for : Leontine Sagan, Carl Froelich
Retrieving (332/590) summary for : Leos Carax
Retrieving (333/590) summary for : Les Norris
Retrieving (334/590) summary for : Lewis Milestone
Retrieving (335/590) summary for : Lloyd Bacon
Retrieving (336/590) summary for : Louis Malle
Retrieving (337/590) summary for : Luc Besson
Retrieving (338/590) summary for : Luc Moullet
Retrieving (339/590) summary for : Luchino Visconti
Retrieving (340/590) summary for : Lucian Pintilie
Retrieving (341/590) summary for : Luigi Comencini
Retrieving (342/590) summary for : Luis Buñuel
Retrieving (343/590) summary for : Lukasz Palkowski
Retrieving (344/590) summary for : Lynne Ramsay
Retrieving (345/590) summary for : M. Night Shyamalan
Retrieving (346/590) summary for : Malgorzata Szumowska
Retrieving (347/590) summary for : Malik Bendjelloul
Retrieving (348/590) summary for : Mamoru Oshii
Retrieving (349/590) summary for : Marcel Carné
Retrieving (350/590) summary for : Marcel Oph

Retrieving (496/590) summary for : Seijun Suzuki
Retrieving (497/590) summary for : Seong-hun Kim
Retrieving (498/590) summary for : Seong-kang Lee
Retrieving (499/590) summary for : Sergei M. Eisenstein
Retrieving (500/590) summary for : Sergei Parajanov
Retrieving (501/590) summary for : Sergio Corbucci
Retrieving (502/590) summary for : Sergio Leone
Retrieving (503/590) summary for : Seth Gordon
Retrieving (504/590) summary for : Shinsuke Sato
Retrieving (505/590) summary for : Shinya Tsukamoto
Retrieving (506/590) summary for : Shin-yeon Won
Retrieving (507/590) summary for : Shion Sono
Retrieving (508/590) summary for : Shohei Imamura
Retrieving (509/590) summary for : Sidney Lumet
Retrieving (510/590) summary for : Simon Hunter
Retrieving (511/590) summary for : Sion Sono
Retrieving (512/590) summary for : Siu-Tung Ching
Retrieving (513/590) summary for : Sogo Ishii
Retrieving (514/590) summary for : Spike Jonze
Retrieving (515/590) summary for : Stanley Donen
Retrieving (516/590

In [7]:
#Stockage dans un fichier pour ne pas avoir à requeter de nouveau
df.to_csv("500 favorite directors_with wikipedia summary.csv",encoding='utf-8', sep=";")

In [2]:
df = pd.read_csv("500 favorite directors_with wikipedia summary.csv", sep=";", low_memory=False, names = ["summary"])

In [3]:
df

,summary
Abdellatif Kechiche,(french: [abdublatif keuiu]; tunisian arabic...
Abel Gance,(french: [guus]; october u november ) was...
Agnès Varda,agnues (french: [auubs vauda]; born may ) is...
Aki Kaurismäki,olavi kaurismueki (finnish: [ucuki uckuurismu...
Akira Kurosawa,"(ueduca ue, march , u september , ) was a j..."
Alain Cavalier,(french: [kavalje]; born september ) is a f...
Alain Resnais,(french: [alubu uubnub]; june u march ) w...
Alain Robbe-Grillet,(french: [a.lubu uub uui.jub]; august u f...
Alan Crosland,"(august , u july , ) was an american stage ..."
Albert Lamorisse,(french: [lamuuis]; january u june ) was ...


In [4]:
#on vire les rows avec un summary vide & on convertit en liste
df = df[df["summary"]!=""]
liste = list(df["summary"])

In [5]:
#seconde passe de nettoyage : on vire les mois qui polluent : les mois
polluting_words =["january","february","march","april","may","june","july","august","september","october","november","december"]
for x in range(len(liste)):
    for polluting_word in polluting_words:
        liste[x]=str(liste[x]).replace(polluting_word,"")

In [6]:
import datetime
#On clusterise N fois, et à chaque fois que X est dans le même groupe que Y on ajouter 1 à la matrice link_strength
nb_cluster = 9
nb_jobs = 100
director_names = list(df.index)
link_strength = pd.DataFrame(data=0,index=director_names,columns = director_names)
for i in range(nb_jobs):
    #On applique la transformation TFIDF
    vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words="english")
    X = vectorizer.fit_transform(liste)
    km = KMeans(n_clusters = nb_cluster ,init="k-means++",max_iter=100,n_init=1,verbose=False)
    km.fit(X)
    cardinal_by_group = np.unique(km.labels_,return_counts=True)[1]
    print(str(i)+":"+str(datetime.datetime.now())+":"+str(np.unique(km.labels_,return_counts=True)))
    #stockage des liens
    for groupe_id in range(nb_cluster):
        df["labels"]=km.labels_
        directors_of_this_group = df[df["labels"]==groupe_id].index
         #on renforce le lien du 1er directeur avec tous les suivants, puis du 2nd avec les suivants, etc.
        for i in range(len(directors_of_this_group)):
            for j in range (i,len(directors_of_this_group)):
                link_strength.loc[[directors_of_this_group[i]], [directors_of_this_group[j]]]+=1
                link_strength.loc[[directors_of_this_group[j]], [directors_of_this_group[i]]]+=1


0:2017-09-01 10:01:24.609439:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 19,  76,  73, 101,  93,  84,  49,  44,  50], dtype=int64))
1:2017-09-01 10:03:03.290083:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 23, 122,  57,  84,  61,  66,  24,  49, 103], dtype=int64))
2:2017-09-01 10:04:48.166082:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 36,  41,  84,  21,  46, 192,  67,  33,  69], dtype=int64))
3:2017-09-01 10:06:58.820555:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 54,  84,  69,  28,  61,   8, 142,  89,  54], dtype=int64))
4:2017-09-01 10:08:55.890251:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 84,  89,  39, 135,  25,  63,  52,  29,  73], dtype=int64))
5:2017-09-01 10:10:48.485691:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 65,  84, 213,  45,  34,  28,  48,  23,  49], dtype=int64))
6:2017-09-01 10:13:16.620164:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 78,  84, 135,  46,  55,  87,   7,  33,  64], dtype=int64))
7:2017-09-01 10:15:03.423273:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), arr

62:2017-09-01 11:56:30.913457:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 64,  74, 174,  30,  84,  17,  48,  70,  28], dtype=int64))
63:2017-09-01 11:58:28.152163:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 89,  38,  46,  84,  59,  45,  21,  75, 132], dtype=int64))
64:2017-09-01 12:00:07.819863:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 47,  84,  19, 157,  27,  61,  36,  32, 126], dtype=int64))
65:2017-09-01 12:02:06.818670:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 53,  84,  65,  43,  64,  55, 173,  35,  17], dtype=int64))
66:2017-09-01 12:04:00.705184:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([ 34,  31, 107,  84,  87,  45, 133,  45,  23], dtype=int64))
67:2017-09-01 12:05:46.416230:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([123,  22,  53,  87,  35,  69,  26,  84,  90], dtype=int64))
68:2017-09-01 12:07:26.256941:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([84, 95, 98, 63, 44, 72, 51, 24, 58], dtype=int64))
69:2017-09-01 12:08:56.334093:(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), arra

In [7]:
#sauvegarde de la matrice de liens
link_strength.to_csv("500 directors_links_matrix.csv",encoding='utf-8', sep=";")

In [475]:
#On récupère les mots les plus courants dans chaque cluster
#Step 1 : récupération du texte total de tous les clusters
text={}
for i,cluster in enumerate(km.labels_):
    oneDoc = liste[i]
    if cluster not in text.keys():
        text[cluster]=oneDoc
    else:
        text[cluster]+=oneDoc
        
#Step 2 : décompte du nb d'occurence de tous les mots et on garde les 100 plus fréquents par cluster
_stopwords = set (stopwords.words("english")+list(punctuation))
keywords={}
counts={}
for cluster in range (nb_cluster):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster]=nlargest(100,freq,key=freq.get)
    counts[cluster]=freq
    
#Step 3 : on ne garde que les mots d'un cluster qui ne sont pas présents dans les autres clusters
unique_keys={}
for cluster in range(nb_cluster):
    other_clusters=list(set(range(nb_cluster))-set([cluster]))
    keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique=set(keywords[cluster])-keys_other_clusters
    unique_keys[cluster]=nlargest(10,unique,key=counts[cluster].get)

unique_keys

{0: ['danish',
  'co-founded',
  'hard',
  'die',
  'hunt',
  'investigator',
  'last',
  'dogme',
  'ordet',
  'space'],
 1: ['drama',
  'picture',
  'nominations',
  'comedy',
  'science',
  'fiction',
  'oscar',
  'dutch',
  'style',
  'actors'],
 2: ['united',
  'released',
  'several',
  'considered',
  'often',
  'direct',
  'produced',
  'war',
  'began',
  'life'],
 3: ['french',
  'la',
  'france',
  'international',
  'le',
  'les',
  'du',
  'documentary',
  'godard',
  'critic'],
 4: ['cannes',
  'german',
  'prize',
  'palme',
  "d'or",
  'international',
  'language',
  'jury',
  'french',
  'kieublowski'],
 5: ['japanese',
  'animator',
  'worked',
  'member',
  'fantasy',
  'polish',
  'dark',
  'python',
  'japan',
  'artist'],
 6: ['time',
  'sound',
  'influential',
  'buufuel',
  'foreign',
  'life',
  'often',
  'international',
  'soviet',
  'sight'],
 7: ['screenplay',
  'original',
  'nomination',
  'whose',
  'writing',
  'dozen',
  'acting',
  'awarded',
  'ti

In [476]:

#Ecriture dans un fichier
df.to_csv("temp.csv",encoding='utf-8', sep=";")